In [4]:
# %%
from tqdm.notebook import tqdm
from PIL import Image
import os

# --- Parâmetros esperados ---
# prefixo = "FDL"
# finalidade = "DL" ou "RL"
# files = lista de caminhos das imagens
# BUCKET_ORIGEM = "storage-unique"
# BUCKET_DESTINO = "storage-curated"
# client = instância do Minio
# conn_pg = conexão ativa com PostgreSQL

print(f"📦 Processando imagens do projeto: {prefixo}")
print(f"🔎 {len(files)} imagens encontradas. Iniciando curadoria...\n")

cur = conn_pg.cursor()

for origem_path in tqdm(files, desc="🎯 Curando imagens", unit="img"):
    try:
        # 🔧 Preparar novo nome
        filename = os.path.basename(origem_path)
        file_ext = os.path.splitext(filename)[1]
        novo_nome = f"arranjado_{filename}"

        # 📥 Abrir imagem e processar
        imagem = Image.open(origem_path).resize((224, 224)).convert("RGB")

        # 🪣 Salvar no bucket de destino
        imagem.save("/tmp/tmp_img.png")
        with open("/tmp/tmp_img.png", "rb") as f:
            client.put_object(
                BUCKET_DESTINO,
                novo_nome,
                f,
                length=os.path.getsize("/tmp/tmp_img.png"),
                content_type="image/png"
            )

        # 📝 Registrar na auditoria
        cur.execute(
            """
            INSERT INTO curation_audit (
                prefix, tipo, finalidade,
                full_path, filename, file_ext,
                novo_nome, bucket_origem, bucket_destino
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (
                prefixo, "imagem", finalidade,
                origem_path, filename, file_ext,
                novo_nome, BUCKET_ORIGEM, BUCKET_DESTINO
            )
        )
        conn_pg.commit()

        print(f"✅ {novo_nome} curado e salvo.")

    except Exception as e:
        conn_pg.rollback()
        print(f"❌ Falha ao processar {origem_path}: {e}")

cur.close()
conn_pg.close()

print("\n🎉 Curadoria leve finalizada.")


NameError: name 'prefixo' is not defined

In [5]:
# %%
import os
import io
from PIL import Image
import ipywidgets as widgets
from IPython.display import display
from tqdm.notebook import tqdm

import psycopg2
import pandas as pd

# --- Conexão com PostgreSQL ---
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "senhasegura"
POSTGRES_HOST = "database-services"
POSTGRES_PORT = "5432"
POSTGRES_DB = "postgres"

conn_pg = psycopg2.connect(
    dbname=POSTGRES_DB,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD,
    host=POSTGRES_HOST,
    port=POSTGRES_PORT
)

# --- Carrega os projetos únicos disponíveis no banco ---
query_projetos = """
    SELECT DISTINCT prefix FROM storage_audit
    WHERE file_ext IN ('.png', '.jpg', '.jpeg')
    ORDER BY prefix;
"""
df_proj = pd.read_sql(query_projetos, conn_pg)
lista_prefixos = df_proj["prefix"].tolist()

# --- Widgets de seleção ---
dropdown_prefix = widgets.Dropdown(
    options=lista_prefixos,
    description='Projeto:',
    style={'description_width': 'initial'}
)

dropdown_finalidade = widgets.Dropdown(
    options=["DL", "RL"],
    description='Finalidade:',
    style={'description_width': 'initial'}
)

botao = widgets.Button(description="Iniciar Curadoria 🚀", button_style="success")

display(dropdown_prefix, dropdown_finalidade, botao)

# --- Ação do botão ---
def processar_curadoria(b):
    prefixo = dropdown_prefix.value
    finalidade = dropdown_finalidade.value
    BUCKET_ORIGEM = "storage-unique"
    BUCKET_DESTINO = "storage-curated"

    # 🔍 Buscar arquivos do projeto
    query_files = """
        SELECT full_path, filename, file_ext
        FROM storage_audit
        WHERE prefix = %s AND file_ext IN ('.png', '.jpg', '.jpeg')
        ORDER BY full_path;
    """
    df_files = pd.read_sql(query_files, conn_pg, params=(prefixo,))
    files = df_files["full_path"].tolist()

    print(f"📦 Projeto selecionado: {prefixo} | Finalidade: {finalidade}")
    print(f"🖼️ {len(files)} imagens encontradas para curadoria...\n")

    cur = conn_pg.cursor()

    for idx, origem_path in enumerate(tqdm(files, desc="🔧 Curando", unit="img")):
        try:
            filename = os.path.basename(origem_path)
            file_ext = os.path.splitext(filename)[1]
            novo_nome = f"arranjado_{filename}"

            # 🖼️ Obter imagem do MinIO
            response = client.get_object(BUCKET_ORIGEM, origem_path)
            imagem = Image.open(io.BytesIO(response.read()))
            imagem_curada = imagem.resize((224, 224)).convert("RGB")

            # 💾 Salvar temporariamente
            temp_path = "/tmp/temp_curado.png"
            imagem_curada.save(temp_path)

            with open(temp_path, "rb") as f:
                client.put_object(
                    BUCKET_DESTINO,
                    novo_nome,
                    f,
                    length=os.path.getsize(temp_path),
                    content_type="image/png"
                )

            # 📝 Auditoria
            cur.execute("""
                INSERT INTO curation_audit (
                    prefix, tipo, finalidade,
                    full_path, filename, file_ext,
                    novo_nome, bucket_origem, bucket_destino
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                prefixo, "imagem", finalidade,
                origem_path, filename, file_ext,
                novo_nome, BUCKET_ORIGEM, BUCKET_DESTINO
            ))
            conn_pg.commit()

        except Exception as e:
            conn_pg.rollback()
            print(f"❌ Falha em {origem_path}: {e}")

    cur.close()
    print("\n🎉 Curadoria finalizada com sucesso.")

botao.on_click(processar_curadoria)


/tmp/ipykernel_961154/1080829765.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_proj = pd.read_sql(query_projetos, conn_pg)


DatabaseError: Execution failed on sql '
    SELECT DISTINCT prefix FROM storage_audit
    WHERE file_ext IN ('.png', '.jpg', '.jpeg')
    ORDER BY prefix;
': column "file_ext" does not exist
LINE 3:     WHERE file_ext IN ('.png', '.jpg', '.jpeg')
                  ^


In [6]:
# %%
from sqlalchemy import create_engine, MetaData, Table, Column
from sqlalchemy import Integer, Text, String, TIMESTAMP
from sqlalchemy.sql import func
from sqlalchemy import inspect

# --- Configurações de conexão PostgreSQL ---
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "senhasegura"
POSTGRES_HOST = "database-services"
POSTGRES_PORT = "5432"
POSTGRES_DB = "postgres"

DATABASE_URL = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
engine = create_engine(DATABASE_URL)
metadata = MetaData()

# --- Verificação de existência ---
inspector = inspect(engine)
existing_tables = inspector.get_table_names()

if "staging_audit" not in existing_tables:
    staging_audit = Table(
        "staging_audit",
        metadata,
        Column("id", Integer, primary_key=True, autoincrement=True),
        Column("timestamp", TIMESTAMP, server_default=func.now()),
        Column("prefix", Text, nullable=False),
        Column("tipo", Text, nullable=False),
        Column("finalidade", Text, nullable=False),
        Column("full_path", Text, nullable=False),
        Column("filename", Text, nullable=False),
        Column("file_ext", String(10), nullable=False),
        Column("novo_nome", Text, nullable=False),
        Column("bucket_origem", Text, nullable=False),
        Column("bucket_destino", Text, nullable=False),
        Column("status", String(20), nullable=False, default="curado"),
        Column("comentario", Text, nullable=True)
    )

    metadata.create_all(engine)
    print("✅ Tabela `staging_audit` criada com sucesso.")
else:
    print("ℹ️ Tabela `staging_audit` já existe. Nenhuma alteração foi feita.")


✅ Tabela `staging_audit` criada com sucesso.


In [7]:
# %%
import os
import io
from PIL import Image
import ipywidgets as widgets
from IPython.display import display
from tqdm.notebook import tqdm
import pandas as pd

from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from datetime import datetime

# --- Configurações de conexão ---
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "senhasegura"
POSTGRES_HOST = "database-services"
POSTGRES_PORT = "5432"
POSTGRES_DB = "postgres"

DATABASE_URL = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

# --- Listar prefixos de projetos com imagens ---
query_prefixos = text("""
    SELECT DISTINCT prefix FROM storage_audit
    WHERE filename ILIKE '%.png' OR filename ILIKE '%.jpg' OR filename ILIKE '%.jpeg'
    ORDER BY prefix
""")
prefixos = [row[0] for row in session.execute(query_prefixos).fetchall()]

# --- Widgets interativos ---
dropdown_prefix = widgets.Dropdown(
    options=prefixos,
    description='Projeto:',
    style={'description_width': 'initial'}
)

dropdown_finalidade = widgets.Dropdown(
    options=["DL", "RL"],
    description='Finalidade:',
    style={'description_width': 'initial'}
)

botao = widgets.Button(description="Iniciar Curadoria 🚀", button_style="success")
display(dropdown_prefix, dropdown_finalidade, botao)

# --- Função principal ---
def processar_curadoria(btn):
    prefixo = dropdown_prefix.value
    finalidade = dropdown_finalidade.value
    BUCKET_ORIGEM = "storage-unique"
    BUCKET_DESTINO = "storage-curated"

    print(f"📦 Projeto: {prefixo} | Finalidade: {finalidade}")

    # 🔍 Recupera imagens do projeto
    query_imgs = text("""
        SELECT full_path, filename FROM storage_audit
        WHERE prefix = :prefix
          AND (filename ILIKE '%.png' OR filename ILIKE '%.jpg' OR filename ILIKE '%.jpeg')
        ORDER BY full_path
    """)
    result = session.execute(query_imgs, {"prefix": prefixo}).fetchall()
    files = [{"full_path": r[0], "filename": r[1]} for r in result]

    print(f"🖼️ {len(files)} imagens encontradas. Iniciando curadoria...\n")

    for item in tqdm(files, desc="🧪 Curando imagens", unit="img"):
        try:
            origem_path = item["full_path"]
            filename = item["filename"]
            file_ext = os.path.splitext(filename)[1]
            novo_nome = f"arranjado_{filename}"

            # 📥 Obter imagem do MinIO
            response = client.get_object(BUCKET_ORIGEM, origem_path)
            imagem = Image.open(io.BytesIO(response.read()))
            imagem_curada = imagem.resize((224, 224)).convert("RGB")

            # 📝 Salvar em staging audit
            stmt = text("""
                INSERT INTO staging_audit (
                    timestamp, prefix, tipo, finalidade,
                    full_path, filename, file_ext,
                    novo_nome, bucket_origem, bucket_destino,
                    status, comentario
                ) VALUES (
                    :timestamp, :prefix, 'imagem', :finalidade,
                    :full_path, :filename, :file_ext,
                    :novo_nome, :bucket_origem, :bucket_destino,
                    :status, :comentario
                )
            """)
            session.execute(stmt, {
                "timestamp": datetime.now(),
                "prefix": prefixo,
                "finalidade": finalidade,
                "full_path": origem_path,
                "filename": filename,
                "file_ext": file_ext,
                "novo_nome": novo_nome,
                "bucket_origem": BUCKET_ORIGEM,
                "bucket_destino": BUCKET_DESTINO,
                "status": "curado",
                "comentario": ""
            })
            session.commit()

        except Exception as e:
            session.rollback()
            print(f"❌ Erro ao processar {origem_path}: {e}")

    print("\n✅ Curadoria leve registrada com sucesso na `staging_audit`.")

botao.on_click(processar_curadoria)


Dropdown(description='Projeto:', options=('FDL',), style=DescriptionStyle(description_width='initial'), value=…

Dropdown(description='Finalidade:', options=('DL', 'RL'), style=DescriptionStyle(description_width='initial'),…

Button(button_style='success', description='Iniciar Curadoria 🚀', style=ButtonStyle())

🧪 Curando imagens:   0%|          | 0/23857 [00:00<?, ?img/s]